In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install mtcnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.3 MB/s eta 0:00:00


In [3]:
import cv2 as cv
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from torchvision import transforms
from torchvision.models import resnet34
import torch
from mtcnn.mtcnn import MTCNN
detector = MTCNN()

In [4]:
# Load the ResNet model
resnet_model = resnet34(pretrained=True)
resnet_model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 88.1MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
from mtcnn.mtcnn import MTCNN
class FACELOADING:
    def __init__(self, directory):
        self.directory = directory
        self.target_size = (224, 224)
        self.X = []
        self.Y = []
        self.detector = MTCNN()
        self.extracted_info = []
        self.unextracted_images = []

    def extract_face(self, filename):
        img = cv.imread(filename)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        try:
            faces = self.detector.detect_faces(img)
            if faces:
                x, y, w, h = faces[0]['box']
                x, y = abs(x), abs(y)
                face = img[y:y + h, x:x + w]
                face_arr = cv.resize(face, self.target_size)

                if face_arr is not None:
                    return face_arr, None
                else:
                    return None, "Face extraction failed"

            else:
                return None, "No face detected"

        except Exception as e:
            return None, f"Face extraction error: {str(e)}"

    def load_faces(self, dir):
        total_images = len(os.listdir(dir))
        extracted_faces = 0
        FACES = []
        for im_name in os.listdir(dir):
            try:
                path = os.path.join(dir, im_name)
                single_face, error = self.extract_face(path)
                if single_face is not None:
                    FACES.append(single_face)
                    extracted_faces += 1
                else:
                    file_name = os.path.basename(path)
                    label_unextracted = os.path.dirname(path)
                    self.unextracted_images.append({'Label unextracted': label_unextracted, 'File_name': file_name, 'Error': error})
            except Exception as e:
                pass
        self.extracted_info.append({'Label': dir, 'Total_images': total_images, 'Extracted_faces': extracted_faces})
        return FACES

    def load_classes(self):
        for sub_dir in os.listdir(self.directory):
            path = os.path.join(self.directory, sub_dir)
            FACES = self.load_faces(path)
            labels = [sub_dir for _ in range(len(FACES))]
            print(f"Loaded successfully: {len(labels)}")
            self.X.extend(FACES)
            self.Y.extend(labels)
        return np.asarray(self.X), np.asarray(self.Y)


In [6]:
# Load images and labels from folder
faceloading = FACELOADING("/content/drive/MyDrive/LVTN/face_recognition_Resnet/dataset")

X, Y = faceloading.load_classes()

1/1 [==============================] - 0s 33ms/step
Loaded successfully: 150
1/1 [==============================] - 0s 27ms/step
Loaded successfully: 30
1/1 [==============================] - 0s 30ms/step
Loaded successfully: 30
1/1 [==============================] - 0s 35ms/step
Loaded successfully: 30
1/1 [==============================] - 0s 33ms/step
Loaded successfully: 30
1/1 [==============================] - 0s 41ms/step
Loaded successfully: 30


In [7]:
plt.figure(figsize=(60,60))
for num,image in enumerate(X):
    ncols = 8
    nrows = len(Y)//ncols + 1
    plt.subplot(nrows,ncols,num+1)
    plt.imshow(image)
    plt.axis('off')

Output hidden; open in https://colab.research.google.com to view.

In [8]:
# Image preprocessing and embedding into feature space using ResNet model
EMBEDDED_X = []
X_np = np.array(X)

def get_embedding_resnet(face_img, resnet_model):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Convert PIL Image to PyTorch tensor
    face_tensor = transform(face_img).unsqueeze(0)

    with torch.no_grad():
        embedding = resnet_model(face_tensor)

    return embedding.flatten().numpy()

for img in X_np:
    img_pil = Image.fromarray((img * 255).astype(np.uint8))
    embedding = get_embedding_resnet(img_pil, resnet_model)
    EMBEDDED_X.append(embedding)

EMBEDDED_X = np.asarray(EMBEDDED_X)

In [9]:
# Convert labels to numbers
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)

In [10]:
# Divide the data set into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(EMBEDDED_X, Y, shuffle=True, random_state=17)

In [11]:
# Train the SVM model
X_train_flatten = X_train
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train_flatten, Y_train)

SVC(kernel='linear', probability=True)

In [12]:
# Evaluate the model on the training and test sets
ypreds_train = svm_model.predict(X_train)
ypreds_test = svm_model.predict(X_test)

In [13]:
train_accuracy = accuracy_score(Y_train, ypreds_train)
test_accuracy = accuracy_score(Y_test, ypreds_test)

In [14]:
print(f"Training Accuracy:
{train_accuracy}")
print(f"Testing Accuracy:
 {test_accuracy}")

Training Accuracy: 1.0
Testing Accuracy: 0.9466666666666667


In [15]:
import os
import cv2 as cv
import numpy as np
import warnings
import time
from PIL import Image
from torchvision import transforms
from sklearn.exceptions import DataConversionWarning

warnings.filterwarnings("ignore", category=DataConversionWarning)

test_folder = "/content/drive/MyDrive/LVTN/face_recognition_Resnet/test"
predictions = []

for filename in os.listdir(test_folder):
    start_time = time.time()
    test_image = cv.imread(os.path.join(test_folder, filename))
    test_image = cv.cvtColor(test_image, cv.COLOR_BGR2RGB)
    x, y, w, h = detector.detect_faces(test_image)[0]['box']
    face_image = test_image[y:y+h, x:x+w]
    face_image = cv.resize(face_image, (224, 224))

    img_pil = Image.fromarray((face_image * 255).astype(np.uint8))

    test_embedding = get_embedding_resnet(img_pil, resnet_model)

    ypred = svm_model.predict([test_embedding])

    predicted_name = encoder.inverse_transform([ypred])[0]
    accuracy = svm_model.predict_proba([test_embedding]).max() * 100

    end_time = time.time()
    process_time = end_time - start_time
    predictions.append((filename, predicted_name, accuracy, process_time))




1/1 [==============================] - 0s 32ms/step


In [16]:
from sklearn.metrics import accuracy_score

true_labels = [' '.join(filename.split(' ')[:-1]) for filename, _, _, _ in predictions]
predicted_labels = [predicted_name for _, predicted_name, _, _ in predictions]

accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy of the Test set is: {accuracy}")

Accuracy of the Test set is: 0.78


In [17]:
total_process_time = sum(process_time for _, _, _, process_time in predictions)
average_process_time = total_process_time / len(predictions) if len(predictions) > 0 else 0

print(f"Average processing time per image: {average_process_time:.4f} seconds")

Average processing time per image: 4.9473 seconds


In [18]:
from prettytable import PrettyTable
table1 = PrettyTable()
table2 = PrettyTable()

# Define field names for both tables
field_names = ["Image", "Predicted", "Accuracy", "Processing Speed", "Result"]

# Add field names to tables
table1.field_names = field_names
table2.field_names = field_names

for filename, predicted_name, accuracy, process_time in predictions:
    true_label = ' '.join(filename.split(' ')[:-1])

    # Define result based on comparison of true_label and predicted_name
    result = "Yes" if true_label == predicted_name else "No"

    # Prepare data for tables based on result
    table_data = [filename, predicted_name, f"{accuracy:.2f}%",
                  f"{process_time:.4f} sec", result]

    if result == "Yes":
        table1.add_row(table_data)
    else:
        table2.add_row(table_data)

# Set title for Table 1
table1.title = "List of correctly predicted images"
if len(table1._rows) > 0:
    print(table1)
else:
    print("There are no correct predictions")
print('\n')

table2.title = "List of incorrectly predicted images"
if len(table2._rows) > 0:
    print(table2)
else:
    print("There are no wrong predictions")

+-----------------------------------------------------------------------+
|                   List of correctly predicted images                  |
+-------------------+------------+----------+------------------+--------+
|       Image       | Predicted  | Accuracy | Processing Speed | Result |
+-------------------+------------+----------+------------------+--------+
|  Truong An 25.jpg | Truong An  |  70.95%  |    6.0481 sec    |  Yes   |
|  Truong An 36.jpg | Truong An  |  61.76%  |    5.5804 sec    |  Yes   |
|  Truong An 47.jpg | Truong An  |  87.12%  |    4.4557 sec    |  Yes   |
|  Truong An 34.jpg | Truong An  |  65.54%  |    5.7937 sec    |  Yes   |
|  Truong An 30.jpg | Truong An  |  99.33%  |    5.6143 sec    |  Yes   |
|  Truong An 39.jpg | Truong An  |  82.83%  |    3.7749 sec    |  Yes   |
|  Truong An 46.jpg | Truong An  |  88.46%  |    4.2791 sec    |  Yes   |
|  Truong An 33.jpg | Truong An  |  98.16%  |    4.8143 sec    |  Yes   |
|  Truong An 22.jpg | Truong An  |  96

In [19]:
total_accuracy = 0
num_correct_predictions = 0

for row in table1._rows:
    accuracy_str = row[field_names.index("Accuracy")]
    accuracy = float(accuracy_str[:-1])
    total_accuracy += accuracy
    num_correct_predictions += 1

average_accuracy = total_accuracy / num_correct_predictions if num_correct_predictions > 0 else 0
print(f"Average accuracy of correctly predicted images of the Resnet model: {average_accuracy:.2f}%")


Average accuracy of correctly predicted images of the Resnet model: 80.37%
